In [1]:
from flask import Flask
from flask_cors import CORS, cross_origin
from draughts import DraughtsBoard, DraughtsPiece
from chess import ChessBoard, ChessPiece
from minimax import best_move
from base import Player
import json

In [2]:
board_types = {'draughts': DraughtsBoard, 'chess': ChessBoard}
piece_types = {'draughts': DraughtsPiece, 'chess': ChessPiece}

In [3]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

def json_to_board(game, board_position_json):
    
    board_position = json.loads(board_position_json)
    
    board_position['player_to_move'] = Player[board_position['player_to_move']]
    
    board_type = board_types[game]
    piece_type = piece_types[game]
    
    board_position['position'] = {(int(sq[1]), int(sq[2])): piece_type[piece] for sq, piece in board_position['position'].items()}

    pm = board_position['previous_move']
    if pm != "none":
        board_position['previous_move'] = ( (int(pm[1]), int(pm[2])), (int(pm[4]), int(pm[5])) )
        
    if board_position.get('move'):
        del(board_position['move'])

    board = board_type.from_dict(board_position)
#    print(board_position_json)
    return board
    
@app.route('/<game>/initial_position')
@cross_origin()
def initial_position(game):
    board_type = board_types[game]
    b = board_type()
    return b.to_json()

@app.route('/<game>/pos/<position_json>')
@cross_origin()
def make_move_given_position(game, position_json):
    "Given a position, find the best move and return the new board with that move played."

    board = json_to_board(game, position_json)

    move = best_move(board, 2)
    new_board = board.make_move(move)
    
    ret = new_board.to_json()
    
    return ret

@app.route('/<game>/move/<pos_and_move_json>')
@cross_origin()
def play_move(game, pos_and_move_json):
    """
    Given a position and a move, play that move and return the new board.
    If the move is illegal, just return the original position.
    The move is in the format S13_S34
    """    
    board = json_to_board(game, pos_and_move_json)
    
#     for sq, piece in board.position.items():
#         print(sq, type(piece))
    
    move = json.loads(pos_and_move_json)['move']
           
    move = ((int(move[1]), int(move[2])), (int(move[5]), int(move[6])))
   
    legal_moves = board.legal_moves()

    if move in legal_moves:
        new_board = board.make_move(move)
    else:
        print(move, ' is not legal.')
        new_board = board
    
    ret = new_board.to_json()
    return ret

In [4]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Sep/2020 19:48:47] "GET /chess/initial_position HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2020 19:48:49] "GET /chess/move/%7B%22position%22:%7B%22S70%22:%22BR%22,%22S71%22:%22BN%22,%22S72%22:%22BB%22,%22S73%22:%22BQ%22,%22S74%22:%22BK%22,%22S75%22:%22BB%22,%22S76%22:%22BN%22,%22S77%22:%22BR%22,%22S60%22:%22BP%22,%22S61%22:%22BP%22,%22S62%22:%22BP%22,%22S63%22:%22BP%22,%22S64%22:%22BP%22,%22S65%22:%22BP%22,%22S66%22:%22BP%22,%22S67%22:%22BP%22,%22S50%22:%22E%22,%22S51%22:%22E%22,%22S52%22:%22E%22,%22S53%22:%22E%22,%22S54%22:%22E%22,%22S55%22:%22E%22,%22S56%22:%22E%22,%22S57%22:%22E%22,%22S40%22:%22E%22,%22S41%22:%22E%22,%22S42%22:%22E%22,%22S43%22:%22E%22,%22S44%22:%22E%22,%22S45%22:%22E%22,%22S46%22:%22E%22,%22S47%22:%22E%22,%22S30%22:%22E%22,%22S31%22:%22E%22,%22S32%22:%22E%22,%22S33%22:%22E%22,%22S34%22:%22E%22,%22S35%22:%22E%22,%22S36%22:%22E%22,%22S37%22:%22E%22,%22S20%22:%22E%22,%22S21%22:%22E%22,%22S22%22:%

{(7, 0): <ChessPiece.BR: ('B', 0, 5, 'Rook')>, (7, 1): 'WB', (7, 2): <ChessPiece.BB: ('B', 0, 3, 'Bishop')>, (7, 3): <ChessPiece.BQ: ('B', 0, 9, 'Queen')>, (7, 4): <ChessPiece.BK: ('B', 0, 999, 'King')>, (7, 5): <ChessPiece.BB: ('B', 0, 3, 'Bishop')>, (7, 6): <ChessPiece.BN: ('B', 0, 3, 'Knight')>, (7, 7): <ChessPiece.BR: ('B', 0, 5, 'Rook')>, (6, 0): <ChessPiece.BP: ('B', 0, 1, 'Pawn')>, (6, 1): <ChessPiece.BP: ('B', 0, 1, 'Pawn')>, (6, 2): <ChessPiece.E: (None, None, None, None)>, (6, 3): <ChessPiece.BP: ('B', 0, 1, 'Pawn')>, (6, 4): <ChessPiece.BP: ('B', 0, 1, 'Pawn')>, (6, 5): <ChessPiece.BP: ('B', 0, 1, 'Pawn')>, (6, 6): <ChessPiece.BP: ('B', 0, 1, 'Pawn')>, (6, 7): <ChessPiece.BP: ('B', 0, 1, 'Pawn')>, (5, 0): <ChessPiece.E: (None, None, None, None)>, (5, 1): <ChessPiece.E: (None, None, None, None)>, (5, 2): <ChessPiece.BN: ('B', 0, 3, 'Knight')>, (5, 3): <ChessPiece.E: (None, None, None, None)>, (5, 4): <ChessPiece.E: (None, None, None, None)>, (5, 5): <ChessPiece.E: (None, None